In [1]:
import pandas as pd 
import seaborn as sns 
import numpy as np 
import matplotlib.pyplot as plt
import mysql.connector

In [2]:
def make_connection_with_db():
    connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="wp_ecommerce")
    cursor = connection.cursor(dictionary=True)
    return connection,cursor 


In [3]:
def get_product_name_from_id(product_id):
    product_name = 'Not Found' # Initialize with default

    try:
        connection, cursor = make_connection_with_db()

        # Check if connection was successful
        if connection is None or cursor is None:
            print("Database connection failed.")
            return product_name

        # Correct SQL query syntax for parameter binding: use %s
        sql = 'SELECT wp_posts.post_title as product_title FROM wp_posts WHERE wp_posts.ID = %s;'

        # Pass parameters as a tuple. mysql.connector handles the escaping.
        cursor.execute(sql, (product_id,))
        results = cursor.fetchall()

        if results: # Check if the list is not empty
            product_name = results[0]['product_title']

    except mysql.connector.Error as err:
        print(f"Database error: {err}. Could not retrieve product name for ID: {product_id}")
        # Optionally log the error details
    except Exception as e:
        print(f"An unexpected error occurred: {e}. Could not retrieve product name for ID: {product_id}")
        # Optionally log the error details
    finally:
        # Ensure cursor and connection are closed in all cases
        if cursor:
            cursor.close()
        if connection:
            connection.close()

    return product_name 

In [4]:
get_product_name_from_id(56053)

'بوط رياضي نسائي-أزرق داكن-ZARA'

In [5]:
get_product_name_from_id(560550)

'Not Found'

In [6]:
def build_dataframe_associated_products():
    # Initialize df outside the try block so it's always available to return
    df = pd.DataFrame(columns=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    connection = None # Initialize connection
    cursor = None     # Initialize cursor

    try:
        connection, cursor = make_connection_with_db()

        # Check if connection was successful
        if connection is None or cursor is None:
            print("Database connection failed.")
            return df # Corrected: return df instead of product_name

        # Correct SQL query syntax for parameter binding: use %s
        sql = 'SELECT * FROM `wp_wc_order_stats` order by order_id ;'

        cursor.execute(sql)
        results = cursor.fetchall()

        if results: # Check if the list is not empty
            for order in results:
                order_id = order['order_id']
                sql = 'SELECT * FROM `wp_wc_order_product_lookup` where order_id=%s ;'
                # Pass parameters as a tuple for secure binding
                cursor.execute(sql, (order_id,))
                results_products = cursor.fetchall()

                products_ids = []
                for product in results_products:
                    product_id = product['product_id']
                    if product_id > 0:
                        products_ids.append(product_id)

                if len(products_ids) > 0:
                    # Your original logic: concatenate a new DataFrame for each order
                    df = pd.concat([df, pd.DataFrame([products_ids])], ignore_index=True)

    except mysql.connector.Error as err:
        # Generalize error message as product_id might not be defined
        print(f"Database error: {err}. Could not retrieve product associations.")
        # Optionally log the error details
    except Exception as e:
        # Generalize error message as product_id might not be defined
        print(f"An unexpected error occurred: {e}. Could not retrieve product associations.")
        # Optionally log the error details
    finally:
        # Ensure cursor and connection are closed in all cases
        if cursor:
            cursor.close()
        if connection:
            connection.close()
    return df

In [7]:
build_dataframe_associated_products()

,0,1,2,3,4,5,6,7,8,9
0,55959,55979,56265,55968,NaN,NaN,NaN,NaN,NaN,NaN
1,55945,55947,56108,56261,55973,NaN,NaN,NaN,NaN,NaN
2,55890,55931,56064,56297,55898,NaN,NaN,NaN,NaN,NaN
3,56007,55982,56152,56286,55981,NaN,NaN,NaN,NaN,NaN
4,55884,55886,56082,56288,55879,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3790,56006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3791,56127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3792,55939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3793,56130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def build_dataframe_associated_products_v2():
    import pandas as pd
    connection = None
    cursor = None
    df = pd.DataFrame()
    all_products_in_orders = []

    try:
        connection, cursor = make_connection_with_db()

        if connection is None or cursor is None:
            print("Database connection failed.")
            return df

        sql = """
            SELECT
                order_id,
                GROUP_CONCAT(product_id ORDER BY product_id ASC) AS product_ids
            FROM
                wp_wc_order_product_lookup
            WHERE
                product_id > 0
            GROUP BY
                order_id
            ORDER BY
                order_id;
        """
        cursor.execute(sql)
        results = cursor.fetchall()

        if results:
            for row in results:
                product_ids_str = row['product_ids']
                if product_ids_str:
                    products_list = [int(p_id) for p_id in product_ids_str.split(',')]
                    all_products_in_orders.append(products_list)

            # الآن نقوم ببناء DataFrame بحيث يكون بنفس الشكل كما في النسخة الأولى
            df = pd.DataFrame(all_products_in_orders)

    except mysql.connector.Error as err:
        print(f"Database error: {err}.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}.")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

    return df


In [9]:
build_dataframe_associated_products_v2()

,0,1,2,3,4
0,55959,55968.0,55979.0,56265.0,NaN
1,55945,55947.0,55973.0,56108.0,56261.0
2,55890,55898.0,55931.0,56064.0,56297.0
3,55981,55982.0,56007.0,56152.0,56286.0
4,55879,55884.0,55886.0,56082.0,56288.0
...,...,...,...,...,...
3790,56006,NaN,NaN,NaN,NaN
3791,56127,NaN,NaN,NaN,NaN
3792,55939,NaN,NaN,NaN,NaN
3793,56130,NaN,NaN,NaN,NaN


In [10]:
df = build_dataframe_associated_products() 

In [11]:


def prepare_transactoins(df): 
    df = df.T 
    transactions = df.apply(lambda x : x.dropna().tolist())
    transactions_list = transactions.values.tolist()
    from mlxtend.preprocessing  import TransactionEncoder 
    te  = TransactionEncoder() 
    te_model = te.fit(transactions_list)
    rows = te_model.transform(transactions_list)

    df_transactions = pd.DataFrame(rows,columns=te_model.columns_)
    return df_transactions

In [12]:
df_transactions =prepare_transactoins(df)

In [50]:
def generate_association_rules(df_transaction,support,confidence):
    from mlxtend.frequent_patterns import apriori 

    frequent_itemsets = apriori(df_transaction,min_support=support,use_colnames=True)

    from mlxtend.frequent_patterns import association_rules 

    rules = association_rules(frequent_itemsets,metric='confidence',min_threshold=confidence)

    return  rules 

rules = generate_association_rules(df_transactions,0.004,0.1)
rules
    

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(56218),(56179),0.020026,0.011594,0.004216,0.210526,18.157895,1.0,0.003984,1.251981,0.964238,0.153846,0.201266,0.287081
1,(56179),(56218),0.011594,0.020026,0.004216,0.363636,18.157895,1.0,0.003984,1.539959,0.956012,0.153846,0.350632,0.287081
2,(56179),(56238),0.011594,0.024242,0.004216,0.363636,15.000000,1.0,0.003935,1.533333,0.944282,0.133333,0.347826,0.268775
3,(56238),(56179),0.024242,0.011594,0.004216,0.173913,15.000000,1.0,0.003935,1.196491,0.956522,0.133333,0.164223,0.268775
4,(56193),(56228),0.021080,0.021080,0.004216,0.200000,9.487500,1.0,0.003772,1.223650,0.913863,0.111111,0.182773,0.200000
5,(56228),(56193),0.021080,0.021080,0.004216,0.200000,9.487500,1.0,0.003772,1.223650,0.913863,0.111111,0.182773,0.200000
6,(56228),(56238),0.021080,0.024242,0.004480,0.212500,8.765625,1.0,0.003969,1.239057,0.904996,0.109677,0.192935,0.198641
7,(56238),(56228),0.024242,0.021080,0.004480,0.184783,8.765625,1.0,0.003969,1.200808,0.907928,0.109677,0.167227,0.198641


In [51]:
rules = generate_association_rules(df_transactions,0.003,0.1)
rules
    

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(56218),(56179),0.020026,0.011594,0.004216,0.210526,18.157895,1.0,0.003984,1.251981,0.964238,0.153846,0.201266,0.287081
1,(56179),(56218),0.011594,0.020026,0.004216,0.363636,18.157895,1.0,0.003984,1.539959,0.956012,0.153846,0.350632,0.287081
2,(56179),(56238),0.011594,0.024242,0.004216,0.363636,15.000000,1.0,0.003935,1.533333,0.944282,0.133333,0.347826,0.268775
3,(56238),(56179),0.024242,0.011594,0.004216,0.173913,15.000000,1.0,0.003935,1.196491,0.956522,0.133333,0.164223,0.268775
4,(56228),(56188),0.021080,0.011858,0.003689,0.175000,14.758333,1.0,0.003439,1.197748,0.952317,0.126126,0.165100,0.243056
5,(56188),(56228),0.011858,0.021080,0.003689,0.311111,14.758333,1.0,0.003439,1.421012,0.943429,0.126126,0.296276,0.243056
6,(56193),(56228),0.021080,0.021080,0.004216,0.200000,9.487500,1.0,0.003772,1.223650,0.913863,0.111111,0.182773,0.200000
7,(56228),(56193),0.021080,0.021080,0.004216,0.200000,9.487500,1.0,0.003772,1.223650,0.913863,0.111111,0.182773,0.200000
8,(56193),(56230),0.021080,0.015547,0.003162,0.150000,9.648305,1.0,0.002834,1.158180,0.915657,0.094488,0.136577,0.176695
9,(56230),(56193),0.015547,0.021080,0.003162,0.203390,9.648305,1.0,0.002834,1.228857,0.910510,0.094488,0.186235,0.176695


In [52]:
support = 0.01 
confidence = 0.1 
n=0
steps = 0 

while n < 2000 : 
    support  = support/2
    steps = steps +1 
    rules = generate_association_rules(df_transactions,support,confidence)
    n = rules.shape[0]
    print(f'rules : {n}  , suppurt : {support} , confidence : {confidence}')

print('Finish')    
    

rules : 0  , suppurt : 0.005 , confidence : 0.1
rules : 72  , suppurt : 0.0025 , confidence : 0.1
rules : 351  , suppurt : 0.00125 , confidence : 0.1
rules : 881  , suppurt : 0.000625 , confidence : 0.1
rules : 2115  , suppurt : 0.0003125 , confidence : 0.1
Finish


In [53]:
def predict(rules,items,max_results=6): 
    preds = rules[rules['antecedents'] == items ]
    preds = preds[['consequents','confidence']]
    # Sort the predictions by 'confidence' in descending order
    # Important: .sort_values() returns a *new* DataFrame.
    # You need to assign the result back to 'preds' or use inplace=True.
    preds = preds.sort_values('confidence', ascending=False)

    # Return the top 'max_results'
    return preds.head(max_results)

In [54]:
preds = predict(rules,{56193})
preds

,consequents,confidence
903,(56228),0.2000
911,(56247),0.1625
905,(56230),0.1500
908,(56235),0.1375
897,(56217),0.1250
844,(56183),0.1125


In [55]:
product_id = 56193 
preds = predict(rules,{product_id})

print(f'Item : {get_product_name_from_id(product_id)}')
print('Recommandation:')
# Loop through the consequents in your predictions
for i, consequent_frozenset in enumerate(preds['consequents']):
    # Check if the frozenset is not empty
    if consequent_frozenset:
        # Extract the single product ID from the frozenset
        # We convert it to a list and take the first element, as frozensets are iterable but not indexable directly
        recommended_product_id = list(consequent_frozenset)[0]
        recommended_product_name = get_product_name_from_id(recommended_product_id)
        print(f' {i}    {recommended_product_name}')
    else:
        print(f' {i}    [No consequent item]')


Item : فرن-بني-HP
Recommandation:
 0    غسالة-أسود-LG
 1    غسالة-فضي-SONY
 2    غسالة-أسود-SONY
 3    مصفف شعر-أسود داكن-SONY
 4    غسالة-أسود-SAMSUNG
 5    براد-فضي-PANASONIC


In [48]:
def export_to_db(rules): 
    connection, cursor = make_connection_with_db() 

    # حذف الجدول إذا كان موجودًا
    cursor.execute("DROP TABLE IF EXISTS custom_products_association")

    # إنشاء الجدول
    create_sql = """ 
        CREATE TABLE custom_products_association (
            ID INT(11) NOT NULL AUTO_INCREMENT,
            product_id_in INT(11) NOT NULL,
            post_title_in TEXT NOT NULL,
            product_id_out INT(11) NOT NULL,
            post_title_out TEXT NOT NULL,
            confidence DOUBLE NOT NULL,
            PRIMARY KEY(ID)
        ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """
    cursor.execute(create_sql)
    connection.commit()

    for row in rules:
        antecedents = row.antecedents
        consequents = row.consequents
        confidence = row.confidence* 100

        for product_in in antecedents:
            post_title_in = get_product_name_from_id(product_in)
            for product_out in consequents:
                post_title_out = get_product_name_from_id(product_out)

                # حذف القيم الأضعف إن وجدت
                delete_sql = """ 
                    DELETE FROM custom_products_association 
                    WHERE product_id_in = %s AND product_id_out = %s AND confidence <= %s
                """
                params = (product_in, product_out, confidence)
                cursor.execute(delete_sql, params)

                # التحقق إن كانت العلاقة موجودة بالفعل بقيمة أعلى
                select_sql = """ 
                    SELECT * FROM custom_products_association 
                    WHERE product_id_in = %s AND product_id_out = %s AND confidence >= %s
                """
                cursor.execute(select_sql, params)
                results = cursor.fetchall()

                # إدراج العلاقة إن لم تكن موجودة
                if not results:
                    insert_sql = """
                        INSERT INTO custom_products_association 
                        (product_id_in, post_title_in, product_id_out, post_title_out, confidence)
                        VALUES (%s, %s, %s, %s, %s)
                    """
                    insert_params = (product_in, post_title_in, product_out, post_title_out, confidence)
                    cursor.execute(insert_sql, insert_params)
                    connection.commit()

    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [45]:
import ast

def prepare_rules(df):
    rules = []
    for _, row in df.iterrows():
        try:
            antecedents = ast.literal_eval(row['antecedents']) if isinstance(row['antecedents'], str) else row['antecedents']
            consequents = ast.literal_eval(row['consequents']) if isinstance(row['consequents'], str) else row['consequents']
            rules.append({
                "antecedents": list(antecedents),
                "consequents": list(consequents),
                "confidence": float(row['confidence'])
            })
        except Exception as e:
            print(f"❌ Error parsing row: {row}")
            print(f"   Reason: {e}")
    return rules




In [56]:
def export_to_db_with_logging(rules: pd.DataFrame):
    connection = None
    cursor = None
    try:
        print("🔌 Connecting to the database...")
        connection, cursor = make_connection_with_db()

        if connection is None or cursor is None:
            print("❌ Failed to establish database connection. Aborting export.")
            logging.error("Failed to establish database connection for export.")
            return

        print("🧹 Dropping existing table if it exists...")
        cursor.execute("DROP TABLE IF EXISTS custom_products_association")
        connection.commit() # Commit after DDL statement

        print("🛠️ Creating the table...")
        create_sql = """
            CREATE TABLE custom_products_association (
                ID INT(11) NOT NULL AUTO_INCREMENT,
                product_id_in INT(11) NOT NULL,
                post_title_in TEXT NOT NULL,
                product_id_out INT(11) NOT NULL,
                post_title_out TEXT NOT NULL,
                confidence DOUBLE NOT NULL,
                PRIMARY KEY(ID)
            ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
        """
        cursor.execute(create_sql)
        connection.commit() # Commit after DDL statement

        print(f"🚀 Exporting {len(rules)} rules to the database...")
        # FIX: Correctly iterate through DataFrame rows using .iterrows()
        for index, row_data in rules.iterrows():
            print(f"📦 Processing rule {index + 1} of {len(rules)}...")
            # Access row data using row_data['column_name']
            antecedents = row_data['antecedents']
            consequents = row_data['consequents']
            confidence = row_data['confidence'] * 100

            # Assuming antecedents and consequents are frozensets of product IDs
            # If they can contain multiple items, the loop below handles it.
            for product_in in antecedents:
                post_title_in = get_product_name_from_id(product_in)
                for product_out in consequents:
                    post_title_out = get_product_name_from_id(product_out)

                    print(f"🔁 Checking association: {product_in} ({post_title_in}) ➡ {product_out} ({post_title_out}) (Confidence: {confidence:.2f}%)")

                    # حذف العلاقات الضعيفة (Delete weaker relationships)
                    delete_sql = """
                        DELETE FROM custom_products_association
                        WHERE product_id_in = %s AND product_id_out = %s AND confidence <= %s
                    """
                    try:
                        cursor.execute(delete_sql, (product_in, product_out, confidence))
                        # No commit here, will commit at the end of loop or after insert
                    except Exception as e:
                        logging.error(f"Error deleting old association for ({product_in}, {product_out}): {e}", exc_info=True)
                        print(f"❗ Error deleting: {e}")

                    # التحقق من وجود علاقة أقوى بالفعل (Check for existing stronger relationship)
                    select_sql = """
                        SELECT * FROM custom_products_association
                        WHERE product_id_in = %s AND product_id_out = %s AND confidence >= %s
                    """
                    try:
                        cursor.execute(select_sql, (product_in, product_out, confidence))
                        results = cursor.fetchall()
                    except Exception as e:
                        logging.error(f"Error selecting existing association for ({product_in}, {product_out}): {e}", exc_info=True)
                        print(f"❗ Error selecting: {e}")
                        results = [] # Assume no results on error to attempt insert

                    if not results: # If no stronger association exists
                        insert_sql = """
                            INSERT INTO custom_products_association
                            (product_id_in, post_title_in, product_id_out, post_title_out, confidence)
                            VALUES (%s, %s, %s, %s, %s)
                        """
                        try:
                            cursor.execute(insert_sql, (product_in, post_title_in, product_out, post_title_out, confidence))
                            print("✅ Inserted new association.")
                        except Exception as e:
                            logging.error(f"Error inserting new association for ({product_in}, {product_out}): {e}", exc_info=True)
                            print(f"❗ Error inserting: {e}")
                    else:
                        print("⏩ Skipped (stronger association exists).")
            connection.commit() # Commit after processing each rule (or batch of rules)

        print("✅ Export completed successfully.")

    except Exception as e:
        logging.error("An error occurred during export_to_db_with_logging", exc_info=True)
        if connection:
            connection.rollback() # Rollback changes if an error occurs
        print("❌ An error occurred. Check 'export_errors.log' for details. Changes rolled back.")

    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()
        print("🔒 Connection closed.")

In [57]:
export_to_db_with_logging(rules)


🔌 Connecting to the database...
🧹 Dropping existing table if it exists...
🛠️ Creating the table...
🚀 Exporting 2115 rules to the database...
📦 Processing rule 1 of 2115...
🔁 Checking association: 55868 (بنطال قماش نسائي-بني داكن-H&M) ➡ 55892 (كنزة داخلية نسائي-فضي-Lululemon) (Confidence: 13.04%)
✅ Inserted new association.
📦 Processing rule 2 of 2115...
🔁 Checking association: 55868 (بنطال قماش نسائي-بني داكن-H&M) ➡ 55919 (كنزة صوف نسائي-أبيض-ZARA) (Confidence: 10.87%)
✅ Inserted new association.
📦 Processing rule 3 of 2115...
🔁 Checking association: 56016 (كندرة نسائي-أسود-Cartier) ➡ 55868 (بنطال قماش نسائي-بني داكن-H&M) (Confidence: 20.00%)
✅ Inserted new association.
📦 Processing rule 4 of 2115...
🔁 Checking association: 56038 (حذاء ساق عالي نسائي-بني داكن-Louis Vuitton) ➡ 55868 (بنطال قماش نسائي-بني داكن-H&M) (Confidence: 18.18%)
✅ Inserted new association.
📦 Processing rule 5 of 2115...
🔁 Checking association: 56047 (حذاء منزلي مغلق نسائي-أزرق داكن-Victoria's Secret) ➡ 55868 (بنطا

In [60]:
def start_generate_association(): 
    min_support = 0.001 
    min_confidence = 0.001 
    df = build_dataframe_associated_products()
    transactions_df =prepare_transactoins(df)
    rules = generate_association_rules(transactions_df,min_support,min_confidence)
    export_to_db_with_logging(rules)

In [61]:
start_generate_association()

🔌 Connecting to the database...
🧹 Dropping existing table if it exists...
🛠️ Creating the table...
🚀 Exporting 904 rules to the database...
📦 Processing rule 1 of 904...
🔁 Checking association: 55881 (كنزة كم نسائي-أزرق داكن-Louis Vuitton) ➡ 55868 (بنطال قماش نسائي-بني داكن-H&M) (Confidence: 6.15%)
✅ Inserted new association.
📦 Processing rule 2 of 904...
🔁 Checking association: 55868 (بنطال قماش نسائي-بني داكن-H&M) ➡ 55881 (كنزة كم نسائي-أزرق داكن-Louis Vuitton) (Confidence: 8.70%)
✅ Inserted new association.
📦 Processing rule 3 of 904...
🔁 Checking association: 55868 (بنطال قماش نسائي-بني داكن-H&M) ➡ 55892 (كنزة داخلية نسائي-فضي-Lululemon) (Confidence: 13.04%)
✅ Inserted new association.
📦 Processing rule 4 of 904...
🔁 Checking association: 55892 (كنزة داخلية نسائي-فضي-Lululemon) ➡ 55868 (بنطال قماش نسائي-بني داكن-H&M) (Confidence: 7.79%)
✅ Inserted new association.
📦 Processing rule 5 of 904...
🔁 Checking association: 55868 (بنطال قماش نسائي-بني داكن-H&M) ➡ 55900 (كنزة داخلية نسائي-

In [72]:
def get_recommandation_products_ids (product_id): 
    connection,cursor = make_connection_with_db() 
    sql = """ 
        SELECT * FROM custom_products_association 
        where  product_id_in = %s order by confidence DESC  """
    params = (product_id , )
    cursor.execute(sql,params)
    products_recommandations = pd.DataFrame()
    results = cursor.fetchall() 
    for product in results : 
        obj = {
            'product_id': product['product_id_out'] , 
            'post_title': product['post_title_out'] , 
            'confidence': product['confidence'] , 
        }
        products_recommandations = pd.concat([products_recommandations,pd.DataFrame([obj])],ignore_index=True)
    return products_recommandations

In [75]:
product_id = 56306
print(f'Item : {get_product_name_from_id(product_id)}')
print('Recommandation:')
get_recommandation_products_ids(product_id)

Item : مجموعة العناية بالبشرة-BLVD 13
Recommandation:


,product_id,post_title,confidence
0,56323,ميكاب-Wild Child,21.428571
1,56350,شامبو مغزي للشعر-Boho Bunny,17.857143
2,56347,ميكاب-SHADE,14.285714
3,56358,زيت تصفيف للشعر-URBN Glow,14.285714
4,56303,زيت تصفيف للشعر-NOIR,14.285714
